# Kubernetes

# Kubernetes Version (Current)

In [199]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort
from kubernetesTool import executePeerCmd

## check status peers

In [112]:
from kubernetesTool import executeKubectlCmd
_res = executeKubectlCmd(['get', 'pods', '-o', 'json'], decode='json')

for item in _res['items']:
    labels=item['metadata']['labels']
    #status=item['status']['initContainerStatuses'][0]
    #print(labels,status['state'],status['restartCount'],status['name'],status['ready'])
    print(labels,item['status']['phase'])
    #print(labels)

{'app': 'hl-fabric-data-share', 'pod-template-hash': '6849487f76'} Running
{'app': 'invoice-net-chaincode-external', 'pod-template-hash': '5c56f49cd8'} Pending
{'app': 'lalya-0', 'pod-template-hash': '7c887cffb8'} Running
{'app': 'lalya-1', 'pod-template-hash': '7c76dff8c'} Running
{'app': 'lalya-2', 'pod-template-hash': '5f8b45dbc4'} Running
{'app': 'malya-0', 'pod-template-hash': '758f9c8df4'} Running
{'app': 'malya-1', 'pod-template-hash': '74679f9f57'} Running
{'app': 'malya-2', 'pod-template-hash': '6dc468fc48'} Running
{'app': 'nalya-0', 'pod-template-hash': '5494459f47'} Running
{'app': 'nalya-1', 'pod-template-hash': '69564b5bb8'} Running
{'app': 'nalya-2', 'pod-template-hash': '864b6b7d9c'} Running
{'app': 'oalya-0', 'pod-template-hash': '5858dcf8b4'} Running
{'app': 'orderer-org1-scray-org', 'pod-template-hash': '9dbd9fc68'} Running
{'app': 'palya-0', 'pod-template-hash': '5b9976c77c'} Running
{'app': 'peer2', 'pod-template-hash': '5645f94d7d'} Running
{'app': 'peer42', 'pod-

In [ ]:
_res['items'][3]['metadata']['labels']
_res['items'][0]['status']['phase']

In [ ]:
30489

In [106]:
kubernetespeer(name='invoice-net-chaincode-external').__dict__

Error from server (NotFound): services "invoice-net-chaincode-external" not found
Error from server (NotFound): services "invoice-net-chaincode-external" not found


{'name': 'invoice-net-chaincode-external',
 'pod_name': 'invoice-net-chaincode-external-5c56f49cd8-4qrm5',
 'listen_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-net-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-listen\\\')].nodePort}"\']\' returned non-zero exit status 1.',
 'gossip_port': 'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'get\', \'service\', \'invoice-net-chaincode-external\', \'-o\', \'jsonpath="{.spec.ports[?(@.name==\\\'peer-gossip\\\')].nodePort}"\']\' returned non-zero exit status 1.'}

In [111]:
for item in _res['items']:
    labels=item['metadata']['labels']
    if labels['app'] == 'invoice-net-chaincode-external':
        print(item)

{'apiVersion': 'v1', 'kind': 'Pod', 'metadata': {'creationTimestamp': '2021-10-20T09:53:33Z', 'generateName': 'invoice-net-chaincode-external-5c56f49cd8-', 'labels': {'app': 'invoice-net-chaincode-external', 'pod-template-hash': '5c56f49cd8'}, 'managedFields': [{'apiVersion': 'v1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:labels': {'.': {}, 'f:app': {}, 'f:pod-template-hash': {}}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"d679532b-220d-4451-b511-0c930cceb22a"}': {'.': {}, 'f:apiVersion': {}, 'f:blockOwnerDeletion': {}, 'f:controller': {}, 'f:kind': {}, 'f:name': {}, 'f:uid': {}}}}, 'f:spec': {'f:containers': {'k:{"name":"invoice-net-chaincode-external"}': {'.': {}, 'f:env': {'.': {}, 'k:{"name":"CHAINCODE_ID"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"CHAINCODE_SERVER_ADDRESS"}': {'.': {}, 'f:name': {}, 'f:value': {}}, 'k:{"name":"FABRIC_LOGGING_SPEC"}': {'.': {}, 'f:name': {}, 'f:value': {}}}, 'f:image': {}, 'f:imagePullPolicy': {}, 'f

## addPeer

In [239]:
def addPeer(peername):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'))
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            

### config            
            
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    
def createConfig(peername='',peer_listen_port='',peer_gossip_port=''):
    try:  
        peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=' + peer_host_name,
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=CORE_PEER_ADDRESS=peer0.' + peer_host_name + ':' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + peer_host_name + ':' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)      
    
  
##### execute

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [11]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [12]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## create peers

In [186]:
peers = {}

In [305]:
peers['zalya-1'].__dict__
#peers

{'name': 'zalya-1',
 'pod_name': 'zalya-1-67c54c9d79-2js2r',
 'listen_port': '31202',
 'gossip_port': ''}

In [67]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')

In [187]:
createPeers('zalya',start=0, end=1)

In [304]:
addPeer('zalya-1')

In [115]:
orderer.__dict__

{'name': 'orderer-org1-scray-org',
 'pod_name': 'orderer-org1-scray-org-9dbd9fc68-5lrgq',
 'listen_port': '30081',
 'gossip_port': '',
 'host': 'orderer.example.com',
 'orderer_intern': 7050}

In [ ]:
kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/inform_existing_nodes.sh $ORDERER_IP $CHANNEL_NAME $PEER_NAME $SHARED_FS_HOST $EXT_PEER_IP $PEER_HOST_NAME

In [ ]:
EER_POD_NAME=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
 ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
 ORDERER_PORT=7050
 PEER_PORT=$(kubectl get service $PEER_NAME -o jsonpath="{.spec.ports[?(@.name=='peer-listen')].nodePort}")
 kubectl exec --stdin --tty $PEER_POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $ORDERER_PORT $CHANNEL_NAME $SHARED_FS_HOST $EXT_PEER_IP

In [189]:
#orderer_ip=get_orderer_ip('orderer-org1-scray-org') 
#orderer_hostname='orderer.example.com'
#orderer_port='7050'

orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
#orderer_port=orderer.orderer_intern
orderer_port= orderer.listen_port

channel_name='super53'
shared_fs_host='10.14.128.38:30080'
    
def endorse_and_join(peer):    
    res1 = endorse_peer(peer=peer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host='10.14.128.38:30080')   
    res2  = joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host)
    return res1,res2

import ipywidgets as widgets
out2 = widgets.Output()

with out2:
    for key, value in peers.items():
        endorse_and_join(value)

In [308]:
endorse_and_join(peers['zalya-1'])

zalya-1
/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super53 zalya-1 10.14.128.38:30080 10.15.136.41 zalya-1.kubernetes.research.dev.seeburger.de


2021-11-02 13:08:28.371 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-11-02 13:08:28.419 UTC [cli.common] readBlock -> INFO 002 Received block: 14
2021-11-02 13:08:28.420 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 1
2021-11-02 13:08:28.430 UTC [cli.common] readBlock -> INFO 004 Received block: 1
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   237  100   237    0     0  59250      0 --:--:-- --:--:-- --:--:-- 59250
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847    0     0  100   847      0  84700 --:--:-- --:--:-- --:--:-- 84700
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
1

/mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super53 10.14.128.38:30080 10.15.136.41
/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer_join.sh '10.32.0.5' orderer.example.com 30081 super53 10.14.128.38:30080 10.15.136.41


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   847  100   847    0     0   165k      0 --:--:-- --:--:-- --:--:--  165k
2021-11-02 13:08:31.667 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-11-02 13:08:31.682 UTC [cli.common] readBlock -> INFO 002 Received block: 0
2021-11-02 13:08:31.768 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-11-02 13:08:31.882 UTC [channelCmd] executeJoin -> INFO 002 Successfully submitted proposal to join channel


('FFFFFFFFF\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz\nOK: 29 MiB in 28 packages\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>301 Moved Permanently</title>\n</head><body>\n<h1>Moved Permanently</h1>\n<p>The document has moved <a href="http://10.14.128.38:30080/ca/">here</a>.</p>\n</body></html>\nRRRRRRR\n',
 'fetch http://dl-cdn.alpinelinux.org/alpine/v3.12/main/x86_64/APKINDEX.tar.gz\nfetch http://dl-cdn.alpinelinux.org/alpine/v3.12/community/x86_64/APKINDEX.tar.gz\n(1/1) Installing curl (7.79.1-r0)\nExecuting busybox-1.31.1-r16.trigger\nOK: 29 MiB in 28 packages\n(1/17) Installing fstrm (0.6.0-r1)\n(2/17) Installing krb5-conf (1.0-r2)\n(3/17) Installing libcom_err (1.45.6-r0)\n(4/17) Installing keyutils-libs (1.6.1-r1)\n(5/17) Installing libverto (0.3.1-r1)\n(6/17) Installing krb5-libs (1.18.4-r0)\n(7/17) Installing json-c (0.14-r1)\n(8/1

In [312]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name)


/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super53


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   463  100   463    0     0  15433      0 --:--:-- --:--:-- --:--:-- 15965
2021-11-02 13:14:53.107 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:27256d95e55ff855dffe6685141fcf6dd3530e4859e8621ec3476f9a69065cd5\022\tbasic_1.0" > 
2021-11-02 13:14:53.107 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:27256d95e55ff855dffe6685141fcf6dd3530e4859e8621ec3476f9a69065cd5
2021-11-02 13:14:55.447 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [6adee26bb9bddf7b05604d2f2d3eaabbadb0f5e81556a7a24b5b3cabc10e8bed] committed with status (VALID) at peer0.zalya-1.kubernetes.research.dev.seeburger.de:31202
Error: query failed with status: 500 - failed to invoke backing implementation of 'CheckCommitReadines

'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'exec\', \'-t\', \'zalya-1-67c54c9d79-2js2r\', \'-c\', \'scray-peer-cli\', \'--\', \'/bin/sh\', \'-c\', "/mnt/conf/install_and_approve_cc.sh 10.14.128.38 \'10.32.0.5\' orderer.example.com 30081 super53"]\' returned non-zero exit status 1.'

In [99]:
#out2.outputs[1]['text'].splitlines()

In [ ]:
#!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-5f97c57d44-j5mbs -c scray-orderer-cli -- /bin/sh -c /mnt/conf/orderer/scripts/inform_existing_nodes.sh '10.32.0.5' super40 kalya-0 10.14.128.38:30080 10.15.136.41 kalya-0.kubernetes.research.dev.seeburger.de

In [28]:
name='peer-gossip'
callopt= 'jsonpath=\"{.spec.ports[?(@.name==\'' + name + '\')].nodePort}\"'
callopt

'jsonpath="{.spec.ports[?(@.name==\'peer-gossip\')].nodePort}"'

## install chaincode

In [190]:
#channel_name
#orderer.__dict__
#peers['ralya-0'].pod_name
peers['zalya-0'].__dict__ , channel_name

({'name': 'zalya-0',
  'pod_name': 'zalya-0-6cf64f96b4-vv4mp',
  'listen_port': '31720',
  'gossip_port': ''},
 'super53')

In [324]:
installAndAproveChaincode(peers['zalya-1'],orderer,channel_name,sequence=2)


['/mnt/conf/install_and_approve_cc.sh', '10.14.128.38', "'10.32.0.5'", 'orderer.example.com', '30081', 'super53', '2']
/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super53 2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   463  100   463    0     0  13228      0 --:--:-- --:--:-- --:--:-- 13228
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-11-02 14:54:57.719 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [63ff906c8ba6be775a27bb9be748ad55065a51eb68c633c0df4186a136b6267d] committed with status (VALID) at peer0.zalya-1.kubernetes.research.dev.seeburger.de:31202


"Installed chaincodes on peer:\nPackage ID: basic_1.0:27256d95e55ff855dffe6685141fcf6dd3530e4859e8621ec3476f9a69065cd5, Label: basic_1.0\nChaincode definition for chaincode 'basic', version '1.0', sequence '2' on channel 'super53' approval status by org:\nAdminOrgMSP: false\nzalya-0MSP: false\nzalya-1MSP: true\n"

In [331]:
#commitChaincode(peers['zalya-1'],channel_name,sequence=2)
chaincode_init(peers['zalya-1'],channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super53


2021-11-02 15:12:08.820 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


'[{"Key":"asset1","Record":{"ID":"asset1","owner":"company","buyer":"","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":false,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset2","Record":{"ID":"asset2","owner":"zalya-0MSP","buyer":"","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":false,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset3","Record":{"ID":"asset3","owner":"zalya-0MSP","buyer":"","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":false,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset4","Record":{"ID":"asset4","owner":"x509::CN=User1@zalya-0.kubernetes.research.dev.seeburger.de,OU=c

In [328]:
def installAndAproveChaincode(peer,orderer,channel_name,cc_host='10.14.128.38',sequence=1):
    #!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK
    #strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.orderer_intern), channel_name ]
    strlist = ['/mnt/conf/install_and_approve_cc.sh', cc_host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name,str(sequence) ]
    #print(strlist)
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

def commitChaincode(peer,channel_name,sequence=1):
    strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence) ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

In [327]:
#cc_host chaincode host

for key, peer in peers.items():
    installAndAproveChaincode(peer,orderer,channel_name,sequence=1)

['/mnt/conf/install_and_approve_cc.sh', '10.14.128.38', "'10.32.0.5'", 'orderer.example.com', '30081', 'super53', '2']
/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super53 2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   463  100   463    0     0  13228      0 --:--:-- --:--:-- --:--:-- 13228
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
2021-11-02 14:58:51.542 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [a94d6a4910c00388e77dea4cfb3ccd4e41a401118d99aa1eed0c4b08c9bb40a4] committed with status (VALID) at peer0.zalya-0.kubernetes.research.dev.seeburger.de:31720


['/mnt/conf/install_and_approve_cc.sh', '10.14.128.38', "'10.32.0.5'", 'orderer.example.com', '30081', 'super53', '2']
/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/install_and_approve_cc.sh 10.14.128.38 '10.32.0.5' orderer.example.com 30081 super53 2


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   463  100   463    0     0  46300      0 --:--:-- --:--:-- --:--:-- 46300
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'ApproveChaincodeDefinitionForMyOrg': attempted to redefine uncommitted sequence (2) for namespace basic with unchanged content


In [326]:


#peer = list(peers.values())[0]
commitChaincode(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc_commit.sh super53


Error: query failed with status: 500 - failed to invoke backing implementation of 'CheckCommitReadiness': requested sequence is 1, but new definition must be sequence 2
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'CommitChaincodeDefinition': requested sequence is 1, but new definition must be sequence 2
command terminated with exit code 1


"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'zalya-1-67c54c9d79-2js2r', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/mnt/conf/peer/cc_commit.sh super53']' returned non-zero exit status 1."

In [62]:
peer=kubernetespeer(name='oalya-0')
channel_name='super45'

In [ ]:
peer.name, channel_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/cc-basic-interaction.sh  super43

In [285]:
#Call init method in chaincode
def chaincode_init(peer,channel_name):
    strlist = ['/mnt/conf/peer/cc-basic-interaction.sh', channel_name ]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

chaincode_init(peer,channel_name)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c /mnt/conf/peer/cc-basic-interaction.sh super53


2021-11-02 10:17:42.508 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


'[{"Key":"asset1","Record":{"ID":"asset1","owner":"company","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":true,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset2","Record":{"ID":"asset2","owner":"zalya-0MSP","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":false,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset3","Record":{"ID":"asset3","owner":"zalya-0MSP","hash":0,"invoiceNumber":"0","vat":0,"netto":0,"countryOrigin":"DE","countryReceiver":"DE","received":false,"receivedOrder":false,"sold":false,"claimPaid":false,"claimPaidBy":"","taxExemptionReason":"","taxReceived":false}},{"Key":"asset4","Record":{"ID":"asset4","owner":"x509::CN=User1@zalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=

In [ ]:
externalbuilder/builds/basic_1.0-21ee1256844f847e196af6aec17c5cafa76d8bd652d6ef42c3058e52571030b4/release/chaincode/server/c

In [37]:
peers['nalya-0'].pod_name

'nalya-0-5494459f47-v7msj'

In [42]:
CHANNEL_NAME='super44'
#ORDERER_NAME=orderer.example.com
IP_CC_SERVICE='10.14.128.38'         # Host where the chaincode is running
#PEER_POD=$(kubectl get pod -l app=$PEER_NAME -o jsonpath="{.items[0].metadata.name}")
#ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')
#ORDERER_LISTEN_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
#ORDERER_HOST=orderer.example.com
EXT_PEER_IP="10.14.128.38"
PEER_POD="nalya-0-5494459f47-v7msj"

ORDERER_PORT=$(kubectl get service orderer-org1-scray-org -o jsonpath="{.spec.ports[?(@.name=='orderer-listen')].nodePort}")
ORDERER_PORT=30081
ORDERER_IP=$(kubectl get pods  -l app=orderer-org1-scray-org -o jsonpath='{.items[*].status.podIP}')

kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/install_and_approve_cc.sh $IP_CC_SERVICE $ORDERER_IP $ORDERER_HOST $ORDERER_PORT $CHANNEL_NAME 

SyntaxError: invalid syntax (1930541980.py, line 11)

In [ ]:
#peers.keys()
#[elem for elem in peers.values()]
list(peers.values())[0].__dict__

# invoice functions

In [218]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername):
    return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
    #return 'x509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE,C=DEdx509::CN=User1@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


## invoice usecase

In [358]:
peernames = list(peers.keys())
peernames

['zalya-0', 'zalya-1']

In [369]:
import json

def getHostname(peer):
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname):
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/User1@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset.values())[:-1] + ']' 

def invoke(peer,callopt):
    export_cmd = getExport_MSPCONFIGPATH(getHostname(peer))
    cmd0 = str('\'' + callopt + "\'")
    strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
    cmd = toCmd(strlist)
    print(cmd)  
    output = executePeerCmd(peer.pod_name, export_cmd + '  && ' + cmd)
    return output

def query(peer, callopt):
    try:
        export_cmd = getExport_MSPCONFIGPATH(getHostname(peer))
        cmd0 = str('\'' + callopt + "\'")
        strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
        cmd = toCmd(strlist)
        output = executePeerCmd(peer.pod_name, export_cmd + '  && ' + cmd)
        return json.loads(output)
        #if output[0] == '{':
        #    return json.loads(output)
        #elif output[0] == '[':
        #    return json.loads(output)
        #return cmd
    except Exception as e:
        print(e)
        return cmd
    
def getAllAssets(peer):
    callopt= '{"function":"GetAllAssets","Args":[' + ']}'
    return query(peer,callopt)    
    
def readAsset(peer, asset='asset1'):
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)

def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def createAsset(peer,arglist=''):
    callopt= '{"function":"CreateAsset","Args":' + arglist + '}'
    return invoke(peer,callopt)

def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)

In [334]:
peer  = peers[pernames[0]]        
asset = readAssetTest(peer)    
asset['ID']='asset6'
asset['buyer']='x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.research.dev.seeburger.de,O=zalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
createAsset(peer,getArgList(asset))

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-0.kubernetes.research.dev.seeburger.de/users/User1@zalya-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super53 -n basic -c '{"function":"ReadAssetTest","Args":["asset1"]}'
peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c '{"function":"CreateAsset","Args":["asset6","company","x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.research.dev.seeburger.de,O=zalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE","0","0","0","0","DE","DE","False","False","False","False","","","False"]}'
/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/p

2021-11-02 15:44:47.525 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [333]:
peers

{'zalya-0': <__main__.kubernetespeer at 0x7fd2d45d27f0>,
 'zalya-1': <__main__.kubernetespeer at 0x7fd2bffd91c0>}

In [372]:
getAllAssets(peers[peernames[1]])

/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-1.kubernetes.research.dev.seeburger.de/users/User1@zalya-1.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super53 -n basic -c '{"function":"GetAllAssets","Args":[]}'


[{'Key': 'asset1',
  'Record': {'ID': 'asset1',
   'owner': 'company',
   'buyer': '',
   'hash': 0,
   'invoiceNumber': '0',
   'vat': 0,
   'netto': 0,
   'countryOrigin': 'DE',
   'countryReceiver': 'DE',
   'received': False,
   'receivedOrder': False,
   'sold': False,
   'claimPaid': False,
   'claimPaidBy': '',
   'taxExemptionReason': '',
   'taxReceived': False}},
 {'Key': 'asset2',
  'Record': {'ID': 'asset2',
   'owner': 'zalya-0MSP',
   'buyer': '',
   'hash': 0,
   'invoiceNumber': '0',
   'vat': 0,
   'netto': 0,
   'countryOrigin': 'DE',
   'countryReceiver': 'DE',
   'received': False,
   'receivedOrder': False,
   'sold': False,
   'claimPaid': False,
   'claimPaidBy': '',
   'taxExemptionReason': '',
   'taxReceived': False}},
 {'Key': 'asset3',
  'Record': {'ID': 'asset3',
   'owner': 'zalya-0MSP',
   'buyer': '',
   'hash': 0,
   'invoiceNumber': '0',
   'vat': 0,
   'netto': 0,
   'countryOrigin': 'DE',
   'countryReceiver': 'DE',
   'received': False,
   'received

In [267]:
asset=readAsset(peer, asset='asset1')
asset['sold'] = True
updateAsset(peer, asset)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-0.kubernetes.research.dev.seeburger.de/users/User1@zalya-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'
peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c '{"function":"UpdateAsset","Args":["asset1","company","0","0","0","0","DE","DE","False","False","True","False","","","False"]}'
/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-0.kubernetes.research.dev.seeburger.de/users/User1@zalya-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pe

2021-10-27 14:20:52.190 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


''

In [381]:
asset = readAsset(peers['zalya-1'], asset='asset6')

from facturxWidget import facturx_widgets 

import json
with open('asset.json', 'w', encoding='utf-8') as f:
    json.dump(asset, f, ensure_ascii=False, indent=4)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c echo $HOSTNAME
/home/jovyan/work/usr/bin/kubectl exec -t zalya-1-67c54c9d79-2js2r -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-1.kubernetes.research.dev.seeburger.de/users/User1@zalya-1.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset6"]}'


In [383]:
import json

with open('asset.json') as json_file:
    data = json.load(json_file)

data    

{'ID': 'asset6',
 'owner': 'x509::CN=User1@zalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-0.kubernetes.research.dev.seeburger.de,O=zalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'buyer': 'x509::CN=User1@zalya-1.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-1.kubernetes.research.dev.seeburger.de,O=zalya-1.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE',
 'hash': 0,
 'invoiceNumber': '0',
 'vat': 0,
 'netto': 0,
 'countryOrigin': 'DE',
 'countryReceiver': 'DE',
 'received': False,
 'receivedOrder': False,
 'sold': False,
 'claimPaid': False,
 'claimPaidBy': '',
 'taxExemptionReason': '',
 'taxReceived': False}

In [268]:
readAsset(peer) 

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-0.kubernetes.research.dev.seeburger.de/users/User1@zalya-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode query -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


{'ID': 'asset1',
 'owner': 'company',
 'hash': 0,
 'invoiceNumber': '0',
 'vat': 0,
 'netto': 0,
 'countryOrigin': 'DE',
 'countryReceiver': 'DE',
 'received': False,
 'receivedOrder': False,
 'sold': True,
 'claimPaid': False,
 'claimPaidBy': '',
 'taxExemptionReason': '',
 'taxReceived': False}

In [244]:
callopt= '{"function":"ReadAsset","Args":[' + '\"' + 'asset1' + '\"' + ']}'
cmd0 = str('\'' + callopt + "\'")
strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
cmd = toCmd(strlist)

executePeerCmd(peers[pernames[0]].pod_name, export_cmd + '  && ' + cmd)

/home/jovyan/work/usr/bin/kubectl exec -t zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/sh -c export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/zalya-0.kubernetes.research.dev.seeburger.de/users/User1@zalya-0.kubernetes.research.dev.seeburger.de/msp/  && peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


2021-10-26 13:37:59.439 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 payload:"{\"ID\":\"asset1\",\"owner\":\"company\",\"hash\":0,\"invoiceNumber\":\"0\",\"vat\":0,\"netto\":0,\"countryOrigin\":\"DE\",\"countryReceiver\":\"DE\",\"received\":false,\"receivedOrder\":false,\"sold\":false,\"claimPaid\":false,\"claimPaidBy\":\"\",\"taxExemptionReason\":\"\",\"taxReceived\":false}" 


''

In [ ]:
 CreateAsset(ctx contractapi.TransactionContextInterface, id, owner string,  hash int,
             invoiceNumber string, vat float32, netto float32, countryOrigin string, countryReceiver string, received bool,
        receivedOrder bool, sold bool, claimPaid bool, claimPaidBy string, taxExemptionReason string,taxReceived bool

In [234]:
def getPeer0Address(peer):
    addr='peer0.kubernetes.research.dev.seeburger.de'
    return str(addr + ':' + peer.listen_port)

def readAsset(peer, asset='asset1'):
    #callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    callopt= '{"function":"ReadAsset","Args":[' + '\"' + asset + '\"' + ']}'
    cmd0 = str('\'' + callopt + "\'")
    try:
        #cmd1= 'CORE_PEER_ADDRESS=\"peer0.kubernetes.research.dev.seeburger.de:31866\" && ' 
        #cmd1 = 'CORE_PEER_ADDRESS=\"' + getPeer0Address(peer) + '\" && '
        cmd1 = 'CORE_PEER_MSPCONFIGPATH=' + path + '\" && '
        #cmd2 = ['peer', 'chaincode','query', '-C', channel_name, '-n', 'basic', '-c']
        cmd2 = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name, '-n', 'basic', '-c']
   
        cmd = toCmd(cmd2) + ' ' + cmd0
        print(cmd)
        #output = executePeerCmd(peer.pod_name, cmd1 + cmd)
        #print(output)
        if output[0] == '{':
            return json.loads(output)
        return cmd
    except Exception as e:
        print(e)
        
asset = readAsset(peers[pernames[0]], 'asset1')
executePeerCmd(peers[pernames[0]].pod_name,asset)

peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'
/home/jovyan/work/usr/bin/kubectl exec -it zalya-0-6cf64f96b4-vv4mp -c scray-peer-cli -- /bin/bash -c peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c '{"function":"ReadAsset","Args":["asset1"]}'


Unable to use a TTY - input is not a terminal or the right kind of file
Error: error endorsing invoke: rpc error: code = Unknown desc = Failed evaluating policy on signed data during check policy on channel [super53] with policy [/Channel/Application/Writers]: [implicit policy evaluation failed - 0 sub-policies were satisfied, but this policy requires 1 of the 'Writers' sub-policies to be satisfied] - proposal response: <nil>
command terminated with exit code 1


'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'exec\', \'-it\', \'zalya-0-6cf64f96b4-vv4mp\', \'-c\', \'scray-peer-cli\', \'--\', \'/bin/bash\', \'-c\', \'peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 -n basic -c \\\'{"function":"ReadAsset","Args":["asset1"]}\\\'\']\' returned non-zero exit status 1.'

In [222]:
invoice_id='046'
buyer=pernames[0]

strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem', '-C',channel_name,invoice_id,get_certificate_string(buyer)]
print(toCmd(strlist))
executePeerCmd(peers[pernames[0]].pod_name, toCmd(strlist))

peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 046 x509::CN=User1@zalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-0.kubernetes.research.dev.seeburger.de,O=zalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE


Unable to use a TTY - input is not a terminal or the right kind of file
Error: must supply value for chaincode name parameter
Usage:
  peer chaincode invoke [flags]

Flags:
  -C, --channelID string               The channel on which this command should be executed
      --connectionProfile string       Connection profile that provides the necessary connection information for the network. Note: currently only supported for providing peer connection information
  -c, --ctor string                    Constructor message for the chaincode in JSON format (default "{}")
  -h, --help                           help for invoke
  -I, --isInit                         Is this invocation for init (useful for supporting legacy chaincodes in the new lifecycle)
  -n, --name string                    Name of the chaincode
      --peerAddresses stringArray      The addresses of the peers to connect to
      --tlsRootCertFiles stringArray   If TLS is enabled, the paths to the TLS root cert files of the p

"Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-it', 'zalya-0-6cf64f96b4-vv4mp', '-c', 'scray-peer-cli', '--', '/bin/bash', '-c', 'peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C super53 046 x509::CN=User1@zalya-0.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.zalya-0.kubernetes.research.dev.seeburger.de,O=zalya-0.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE']' returned non-zero exit status 1."

In [ ]:
CHANNEL_ID=$1
ASSET_ID=$2
PRODUCT_BUYER=$3

export PKGID=basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee

export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/$HOSTNAME/users/User1@$HOSTNAME/msp/

peer chaincode invoke -o orderer.example.com:30081 --tls --cafile /tmp/tlsca.example.com-cert.pem -C $CHANNEL_ID -n basic -c '{"function":"CreateAsset","Args":["'${ASSET_ID}'", "'${PRODUCT_BUYER}'"]}'

## invoice usecase (Simple)

In [74]:

pernames = list(peers.keys())
chaincode_add_invoice(peers[pernames[0]],channel_name,'046',pernames[1])

IndexError: list index out of range

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -t galya-0-6cc848b49-pv2dn -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh super13 006 x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US

In [ ]:
get_certificate_string(pernames[1])

#'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

#, 'x509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=USdx509::CN=User1@galya-1.kubernetes.research.dev.seeburger.de,OU=client,L=San Francisco,ST=California,C=US::CN=ca.galya-1.kubernetes.research.dev.seeburger.de,O=galya-1.kubernetes.research.dev.seeburger.de,L=San Francisco,ST=California,C=US'

In [ ]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  $CHANNEL_NAME $INVOICE_ID
def chaincode_read_invoice(peer,channel_name):
    strlist = ['/mnt/conf/peer/get-my-invoices.sh', channel_name]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 
chaincode_read_invoice(peers[pernames[0]],channel_name)

In [ ]:
def chaincode_transfer_invoice(peer,channel_name,invoice_id,new_owner):
    strlist = ['/mnt/conf/peer/transfer_invoice.sh', channel_name,invoice_id,get_certificate_string(new_owner)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

pernames = list(peers.keys())
chaincode_transfer_invoice(peers[pernames[0]],channel_name,'045',pernames[2])

In [ ]:
get_certificate_string(pernames[2])
NEW_OWNER='x509::CN=User1@lalya-2.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.lalya-2.kubernetes.research.dev.seeburger.de,O=lalya-2.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'
kubectl exec --stdin --tty lalya-0-7c887cffb8-fpw7h -c scray-peer-cli -- /bin/sh /mnt/conf/peer/transfer_invoice.sh  super43 045 $NEW_OWNER

In [ ]:
kubectl exec --stdin --tty 'lalya-0-7c887cffb8-fpw7h' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-1-7c76dff8c-pzx78' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045
kubectl exec --stdin --tty 'lalya-2-5f8b45dbc4-52tt7' -c scray-peer-cli -- /bin/sh /mnt/conf/peer/get-my-invoices.sh  super43 045


In [ ]:
peers[pernames[2]].pod_name

## add channel

In [ ]:
orderer.pod_name

In [183]:
!/home/jovyan/work/usr/bin/kubectl exec -t orderer-org1-scray-org-9dbd9fc68-5lrgq -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super53 orderer.example.com 30081

2021-10-22 08:36:42.399 UTC [common.tools.configtxgen] main -> INFO 001 Loading configuration
2021-10-22 08:36:42.429 UTC [common.tools.configtxgen.localconfig] Load -> INFO 002 Loaded configuration: /mnt/conf/orderer/configtx.yaml
2021-10-22 08:36:42.429 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 003 Generating new channel configtx
2021-10-22 08:36:42.432 UTC [common.tools.configtxgen] doOutputChannelCreateTx -> INFO 004 Writing new channel tx
2021-10-22 08:36:42.501 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2021-10-22 08:36:42.575 UTC [cli.common] readBlock -> INFO 002 Expect block, but got status: &{NOT_FOUND}
2021-10-22 08:36:42.583 UTC [channelCmd] InitCmdFactory -> INFO 003 Endorser and orderer connections initialized
2021-10-22 08:36:42.785 UTC [cli.common] readBlock -> INFO 004 Expect block, but got status: &{SERVICE_UNAVAILABLE}
2021-10-22 08:36:42.803 UTC [channelCmd] InitCmdFactory -> INFO 005 Endorser and or

In [ ]:
add_channel('super20')

In [ ]:
name = 'super30'
cmd = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050'])
cmd, orderer.pod_name

In [ ]:
executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')

In [ ]:
executePeerCmd(orderer.pod_name, 'peer channel list', cli='scray-orderer-cli')

In [ ]:
cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh super20 orderer.example.com 7050']
executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

In [ ]:
#cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c','/mnt/conf/orderer/scripts/create_channel.sh super7 orderer.example.com 7050']

#cmd2 = toCmd(['ls','-a'])
cmd2 = toCmd(['/mnt/conf/orderer/scripts/create_channel.sh', 'super7', 'orderer.example.com', '7050'])

cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', 'orderer-org1-scray-org-5f97c57d44-l6d5l', '-c', 'scray-orderer-cli','--', '/bin/sh', '-c',cmd2]


print(toCmd(cmd1))
#subprocess.check_output(cmd1)
subprocess.run(cmd1, stdout = subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec -it orderer-org1-scray-org-5f97c57d44-l6d5l -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh super6 orderer.example.com 7050


In [ ]:
orderer.pod_name

In [ ]:
!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty 'orderer-org1-scray-org-5f97c57d44-j5mbs' -c scray-orderer-cli  -- /bin/sh /mnt/conf/orderer/scripts/create_channel.sh 'super' orderer.example.com '7050'

In [ ]:
# Create Peer

peername='dalya8'
addPeer(peername)
peers[peername].__dict__

In [ ]:
executeKubectlCmd(['get', 'pods','-l', 'app=orderer-org1-scray-org','-o','jsonpath=\'{.items[*].status.podIP}\''])  

In [ ]:
peers[peername].__dict__

In [ ]:
orderer.__dict__

## Export channel configuration

In [ ]:
#https://github.com/scray/scray/blob/feature/k8s-peer/projects/invoice-hyperledger-fabric/containers/README.md#export-data

SHARED_FS_HOST='10.14.128.38:30080'
_result = SHARED_FS_HOST + '/channel/configuration/$CHANNEL_NAME/config.json'

#kubectl exec --stdin --tty $ORDERER_POD -c scray-orderer-cli -- /bin/sh /mnt/conf/orderer/scripts/publish_channel_conf.sh $CHANNEL_NAME $SHARED_FS_HOST

def export_channel_configuration(peer,channel_name,SHARED_FS_HOST):
    strlist = ['/mnt/conf/orderer/scripts/publish_channel_conf.sh', channel_name,SHARED_FS_HOST]
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 

#_peer = peers[list(peers.keys())[0]]
_peer = orderer
channel_name = 'super'
export_channel_configuration(_peer,channel_name,SHARED_FS_HOST)

In [ ]:
#list(peers)[0]


# Enroll users

In [ ]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray/projects/invoice-hyperledger-fabric/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='upload')
    uploadFromLocal(local_path='/home/jovyan/scray/projects/invoice-hyperledger-fabric/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
#createPeerUser(peers['oly0'],'bretten')     

In [ ]:
peer = peers['oly0']
# build local , upload tool to sharepoint 
#uploadWalletToolToSharepoint()
#installPeerWalletTool(peer)

createPeerUser(peer,'bretten')   